<a href="https://colab.research.google.com/github/t1207a/My-start-project/blob/master/6%EB%8B%A8%EC%9B%90_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.datasets import make_moons #make_moons 데이터셋 생성
X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
param_grid = [
              {'max_leaf_nodes': list(range(2,100)),'min_samples_leaf':[2,3,4]}
]
grid_search = GridSearchCV(DecisionTreeClassifier(),param_grid,verbose=1,cv=3,n_jobs=-1)
grid_search.fit(X_train,y_train)

Fitting 3 folds for each of 294 candidates, totalling 882 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 360 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 882 out of 882 | elapsed:    7.6s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid=[{'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,

In [5]:
grid_search.best_params_

{'max_leaf_nodes': 20, 'min_samples_leaf': 2}

In [8]:
from sklearn.metrics import accuracy_score
y_pred = grid_search.predict(X_test)
accuracy_score(y_test,y_pred)

0.8605

In [29]:
# 8번 서브셋 1000개 생성
from sklearn.model_selection import ShuffleSplit
n_trees = 1000
n_instances = 100
mini_sets=[]

ss= ShuffleSplit(n_splits = n_trees, test_size = len(X_train)-n_instances)
for mini_train_index, mini_test_index in ss.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))


In [31]:
from sklearn.base import clone
import numpy as np
forest = [clone(grid_search.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores)

0.7915045

In [32]:
Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)

In [34]:
from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

In [35]:
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.871